<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/phunc20/python/blob/master/math/e/concise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Origin
The story began like this
> A friend of mine on Facebook,
> who is a high school teacher, one day
> posted what one of his pupils asked him:  
> "_Hi Mr., we were taught that the sequence
> $a_{n} = (1 + \frac{1}{n})^{n}$ is a bounded
> increasing sequence converging to
> $e \approx 2.718281828459045$;
> how come Google gives $2.71828203876$, which
> is apparently greater than $e$, when
> one inputs `(1+1/10^9)^(10^9)` in Google search?_ "

One could reply, "_Because floating-point arithmetic is different
from the arithmetic that we are most used to and taught since childhood. And computers adapt floating-point arithmetic._"

But that answer appears **irresponsible** and **non educative**.
I would like to try to establish an answer suitable for
explaining to such pupils.

First, let's try to repeat the same experiment as the pupil did on
Google.

In [10]:
def a(n):
    return (1 + 1/n)**n

In [8]:
# To gain access to the constant e in Python
import math

In [19]:
MAX_POW = 15
for p in range(1, MAX_POW+1):
    n = 10**p
    an = a(n)
    head = f"a(10**{p:2d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    print(head + tail)

a(10** 1) = 2.5937424601000023 < e
a(10** 2) = 2.7048138294215285 < e
a(10** 3) = 2.7169239322355936 < e
a(10** 4) = 2.7181459268249255 < e
a(10** 5) = 2.7182682371922975 < e
a(10** 6) = 2.7182804690957534 < e
a(10** 7) = 2.7182816941320818 < e
a(10** 8) = 2.7182817983473577 < e
a(10** 9) = 2.7182820520115603 > e
a(10**10) = 2.7182820532347876 > e
a(10**11) = 2.71828205335711 > e
a(10**12) = 2.7185234960372378 > e
a(10**13) = 2.716110034086901 < e
a(10**14) = 2.716110034087023 < e
a(10**15) = 3.035035206549262 > e


Good! We do observe the same phenomenon as the pupil.

The output raised at least three questions:
1. The question the pupil raised
1. Why `a(10**13)` suddenly drops to `2.716`?
1. Is `a(n)` closer and closer to `math.e` in floating-point arithmetic?

## Proposed Answer
As the pupil described, the sequence $a_{n} = (1 + \frac{1}{n})^{n}$
is indeed an increasing sequence bounded above converging to $e.$

Since computers could only represent a finit number of numbers,
the base part $1 + \frac{1}{n}$ of $a_{n}$ is subject to **rounding** --
it needs to be either **rounded up** or **down**.

As n gets bigger and bigger, $a_{n}$ gets closer and closer to $e$. What happens when `n = 10**9` could be explained as follows:

> $a_{10^9}$ is quite close to $e$. In the mean time,
> `1 + 1/(10**9)` needs to be **rounded up** to
> a machine-representable number, making it **bigger** than
> $1 + \frac{1}{10^9}$.
> When raised to the same power, we'd expect `a(10**9)` $\gneq a_{10^9} \approx e.$

This answers questions 1 and 2.

In Python, there exists this `decimal` package which could help us
represent better numbers in a way we are more used to.
For out purpose, we could use them to know **whether the base part is
rounded up or down**.

In [12]:
from decimal import Decimal

In [34]:
for p in range(1, MAX_POW+1):
    n = 10**p
    base = 1 + 1/n
    an = a(n)
    theoretical_base = Decimal(1) + Decimal(1)/Decimal(n)
    if theoretical_base > base:
        head = "( down)"
    elif theoretical_base == base:
        head = "(equal)"
    else:
        head = "(   up)"
    body = f" a(10**{p:2d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    print(head + body + tail)

(   up) a(10** 1) = 2.5937424601000023 < e
(   up) a(10** 2) = 2.7048138294215285 < e
( down) a(10** 3) = 2.7169239322355936 < e
( down) a(10** 4) = 2.7181459268249255 < e
(   up) a(10** 5) = 2.7182682371922975 < e
( down) a(10** 6) = 2.7182804690957534 < e
(   up) a(10** 7) = 2.7182816941320818 < e
( down) a(10** 8) = 2.7182817983473577 < e
(   up) a(10** 9) = 2.7182820520115603 > e
(   up) a(10**10) = 2.7182820532347876 > e
(   up) a(10**11) = 2.71828205335711 > e
(   up) a(10**12) = 2.7185234960372378 > e
( down) a(10**13) = 2.716110034086901 < e
( down) a(10**14) = 2.716110034087023 < e
(   up) a(10**15) = 3.035035206549262 > e


We see indeed that
> After `10**9`, those whose final result is **greater than** `math.e`
> have their base part **rounded up** and vice versa.

It is quite obvious that the answer to question 3 is
> No, the `a(n)`'s are not closer and closer to `math.e`.

In [24]:
MAX_POW = 15
for p in range(1, MAX_POW+1):
    n = 10**p
    an = a(n)
    #diff = abs(math.e - an)
    #print(f"abs(a(10**{p:2d}) - e) = {diff}")
    diff = an - math.e
    print(f"a(10**{p:2d}) - e = {diff}")

a(10** 1) - e = -0.12453936835904278
a(10** 2) - e = -0.01346799903751661
a(10** 3) - e = -0.0013578962234515046
a(10** 4) - e = -0.000135901634119584
a(10** 5) - e = -1.359126674760347e-05
a(10** 6) - e = -1.359363291708604e-06
a(10** 7) - e = -1.3432696333026684e-07
a(10** 8) - e = -3.011168736577474e-08
a(10** 9) - e = 2.2355251516614771e-07
a(10**10) - e = 2.2477574246337895e-07
a(10**11) - e = 2.248980650598753e-07
a(10**12) - e = 0.00024166757819266138
a(10**13) - e = -0.002171794372144209
a(10**14) - e = -0.0021717943720220845
a(10**15) - e = 0.31675337809021675


## The Same Code but This Time with `Decimal`

In [28]:
def b(n):
    return (Decimal(1) + Decimal(1)/Decimal(n))**Decimal(n)

In [29]:
for p in range(1, MAX_POW+1):
    n = 10**p
    bn = b(n)
    body = f" b(10**{p:2d}) = {bn}"
    tail = " > e" if bn > math.e else " < e"
    print(body + tail)

 b(10** 1) = 2.5937424601 < e
 b(10** 2) = 2.704813829421526093267194711 < e
 b(10** 3) = 2.716923932235892457383088122 < e
 b(10** 4) = 2.718145926825224864037664675 < e
 b(10** 5) = 2.718268237174489668035064824 < e
 b(10** 6) = 2.718280469319376883819799708 < e
 b(10** 7) = 2.718281692544966271198550226 < e
 b(10** 8) = 2.718281814867636217652977243 < e
 b(10** 9) = 2.718281827099904322376644024 < e
 b(10**10) = 2.718281828323131143949794001 < e
 b(10**11) = 2.718281828445453826218116833 < e
 b(10**12) = 2.718281828457686094446059195 < e
 b(10**13) = 2.718281828458909321268864532 < e
 b(10**14) = 2.718281828459031643951145176 < e
 b(10**15) = 2.718281828459043876219373242 < e


In [32]:
for p in range(1, MAX_POW+1):
    n = 10**p
    bn = b(n)
    diff = bn - Decimal(f"{math.e}")
    print(f"b(10**{p:2d}) - e = {diff}")

b(10** 1) - e = -0.124539368359045
b(10** 2) - e = -0.013467999037518906732805289
b(10** 3) - e = -0.001357896223152542616911878
b(10** 4) - e = -0.000135901633820135962335325
b(10** 5) - e = -0.000013591284555331964935176
b(10** 6) - e = -0.000001359139668116180200292
b(10** 7) - e = -1.35914078728801449774E-7
b(10** 8) - e = -1.3591408782347022757E-8
b(10** 9) - e = -1.359140677623355976E-9
b(10**10) - e = -1.35913856050205999E-10
b(10**11) - e = -1.3591173781883167E-11
b(10**12) - e = -1.358905553940805E-12
b(10**13) - e = -1.35678731135468E-13
b(10**14) - e = -1.3356048854824E-14
b(10**15) - e = -1.123780626758E-15
